## 2. Trial by trial decoding analysis - Likelihood

This notebook uses previously parsed ripple times in the TrialChoiceReplay() table. to make plots for decoded ripple content using either state-space decoder result (2ms time window) vs a likelihood decoder (20 ms time window).

May 6, 2025
Shijie Gu

In [1]:
%reload_ext autoreload
%autoreload 2

In [44]:
import spyglass as nd
import pandas as pd
import os
# ignore datajoint+jupyter async warnings
import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)
warnings.simplefilter('ignore', category=ResourceWarning)

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import logging
import multiprocessing

FORMAT = '%(asctime)s %(message)s'

logging.basicConfig(level='INFO', format=FORMAT, datefmt='%d-%b-%y %H:%M:%S')

from spyglass.common import (Session, IntervalList,IntervalPositionInfo,
                             LabMember, LabTeam, Raw, Session, Nwbfile,
                            Electrode,LFPBand,interval_list_intersect)
from spyglass.common.common_interval import _intersection

#from spyglass.decoding.clusterless import (UnitMarks,
#                                           UnitMarkParameters,UnitMarksIndicatorSelection,
#                                          UnitMarksIndicator)
from spyglass.utils.nwb_helper_fn import get_nwb_copy_filename
from spyglass.common.common_position import IntervalPositionInfo, IntervalPositionInfoSelection
from spyglass.common.common_position import IntervalLinearizedPosition

from spyglass.spikesorting.v0 import (SortGroup, 
                                    SortInterval,
                                    SpikeSortingPreprocessingParameters,
                                    SpikeSortingRecording,SpikeSortingRecordingSelection)

from spyglass.shijiegu.load import load_epoch_data, load_decode
# Here are the analysis tables specific to Shijie Gu
from spyglass.shijiegu.Analysis_SGU import (TrialChoice,
                                   TrialChoiceReplay,
                                   RippleTimes,
                                   DecodeResultsLinear,get_linearization_map,
                                   find_ripple_times,mua_thresholder,sort_replays,
                                   classify_ripples,classify_ripple_content,
                                   load_everything,MUA,
                                   TetrodeNumber,NotExtendedTrialChoiceReplay
                                  )
from spyglass.shijiegu.Analysis_SGU import EpochPos,RippleTimesWithDecode,ExtendedRippleTimesWithDecode,ExtendedRippleTimes,ExtendedTrialChoiceReplay,TrialChoiceReplayTransition

from spyglass.shijiegu.ripple_add_replay import (replay_parser_master,plot_ripples_replays, select_subset_helper)

In [91]:
# input day
nwb_file_name = 'lewis20240109.nwb'
nwb_copy_file_name = get_nwb_copy_filename(nwb_file_name)

### 1. Check there are likelihood decodes

In [92]:
DecodeResultsLinear & {'nwb_file_name': nwb_copy_file_name}

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,classifier_param_name a name for this set of parameters,encoding_set a name for this set of encoding,posterior posterior within that interval (1D)
lewis20240109_.nwb,02_Rev2Session1,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=
lewis20240109_.nwb,02_Rev2Session1,default_decoding_gpu_4armMaze_W20msO10ms,2Dheadspeed_above_4,=BLOB=
lewis20240109_.nwb,04_Rev2Session2,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=
lewis20240109_.nwb,04_Rev2Session2,default_decoding_gpu_4armMaze_W20msO10ms,2Dheadspeed_above_4,=BLOB=
lewis20240109_.nwb,04_Rev2Session2,default_decoding_gpu_4armMaze_W40msO20ms,2Dheadspeed_above_4,=BLOB=
lewis20240109_.nwb,06_Rev2Session3,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=
lewis20240109_.nwb,06_Rev2Session3,default_decoding_gpu_4armMaze_W20msO10ms,2Dheadspeed_above_4,=BLOB=
lewis20240109_.nwb,06_Rev2Session3,default_decoding_gpu_4armMaze_W40msO20ms,2Dheadspeed_above_4,=BLOB=
lewis20240109_.nwb,08_Rev2Session4,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=
lewis20240109_.nwb,08_Rev2Session4,default_decoding_gpu_4armMaze_W20msO10ms,2Dheadspeed_above_4,=BLOB=


In [94]:
# 2. input encoding_set, classifier_param_name
session_name = "02_Rev2Session1"
epoch_num = 2

encoding_set = "2Dheadspeed_above_4"
decoder_lk = "default_decoding_gpu_4armMaze_W20msO10ms"
decoder_ss = "default_decoding_gpu_4armMaze"

key_lk = {'nwb_file_name': nwb_copy_file_name,
          "encoding_set":encoding_set,"interval_list_name": session_name,
          "classifier_param_name": decoder_lk}

key_ss = {'nwb_file_name': nwb_copy_file_name,
          "encoding_set":encoding_set,"interval_list_name": session_name,
          "classifier_param_name": decoder_ss}

print(DecodeResultsLinear & key_lk)
print(DecodeResultsLinear & key_ss)

*nwb_file_name *interval_list *classifier_pa *encoding_set  posterior 
+------------+ +------------+ +------------+ +------------+ +--------+
lewis20240109_ 02_Rev2Session default_decodi 2Dheadspeed_ab =BLOB=    
 (Total: 1)

*nwb_file_name *interval_list *classifier_pa *encoding_set  posterior 
+------------+ +------------+ +------------+ +------------+ +--------+
lewis20240109_ 02_Rev2Session default_decodi 2Dheadspeed_ab =BLOB=    
 (Total: 1)



### 2. Check there are parsed decode results from state space decoder.
We use parsed results (fragmented vs continuous) from the state space decoder.

In [104]:
decode_threshold_method = "MUA_M05SD"
key_ss["decode_threshold_method"] = decode_threshold_method

In [105]:
RippleTimesWithDecode & key_ss

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,classifier_param_name a name for this set of parameters,encoding_set a name for this set of encoding,decode_threshold_method a name for this thresholding method,ripple_times ripple times within that interval
lewis20240109_.nwb,02_Rev2Session1,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_M05SD,=BLOB=


In [106]:
key_ss["decode_threshold_method"] = decode_threshold_method

ripple_times = pd.read_pickle((RippleTimesWithDecode() & key_ss).fetch1("ripple_times"))
log_df = pd.DataFrame((NotExtendedTrialChoiceReplay() & key_ss).fetch1("choice_reward_replay"))

In [108]:
# load lk decoder
decode_lk = load_decode(nwb_copy_file_name, session_name, key_lk["classifier_param_name"], encoding_set)

### 3. plot intervals detected by state space decoder with likelihood decoding result.

In [109]:
# need to get theta_df, ripple_df, neural_df
(epoch_name,_,decode_ss,
 head_speed,head_orientation,linear_position_df,
 lfp_df,theta_df,ripple_df,neural_df,_) = load_epoch_data(nwb_copy_file_name,epoch_num,
                                                          key_ss["classifier_param_name"],
                                                          encoding_set)



epoch name 02_Rev2Session1
epoch_pos_name pos 1 valid times


[2025-05-07 12:30:37,776][WARNING]: Skipped checksum for file with hash: 6cfcf895-2e2b-bb17-c3d1-9422e1618213, and path: /stelmo/nwb/analysis/lewis20240109/lewis20240109_R4LWKFUE59.nwb
07-May-25 12:30:37 Skipped checksum for file with hash: 6cfcf895-2e2b-bb17-c3d1-9422e1618213, and path: /stelmo/nwb/analysis/lewis20240109/lewis20240109_R4LWKFUE59.nwb
[2025-05-07 12:30:37,801][WARNING]: Skipped checksum for file with hash: 6cfcf895-2e2b-bb17-c3d1-9422e1618213, and path: /stelmo/nwb/analysis/lewis20240109/lewis20240109_R4LWKFUE59.nwb
07-May-25 12:30:37 Skipped checksum for file with hash: 6cfcf895-2e2b-bb17-c3d1-9422e1618213, and path: /stelmo/nwb/analysis/lewis20240109/lewis20240109_R4LWKFUE59.nwb
[2025-05-07 12:30:37,809][WARNING]: Skipped checksum for file with hash: 6cfcf895-2e2b-bb17-c3d1-9422e1618213, and path: /stelmo/nwb/analysis/lewis20240109/lewis20240109_R4LWKFUE59.nwb
07-May-25 12:30:37 Skipped checksum for file with hash: 6cfcf895-2e2b-bb17-c3d1-9422e1618213, and path: /stel

In [110]:
# MUA
mua_path=(MUA & {'nwb_file_name': nwb_copy_file_name,
                 'interval_list_name':session_name}).fetch1('mua_trace')
mua_xr = xr.open_dataset(mua_path)
if decode_threshold_method == 'MUA_0SD':
    mua_threshold=(MUA & {'nwb_file_name': nwb_copy_file_name,
                 'interval_list_name':session_name}).fetch1('mean')
elif decode_threshold_method == 'MUA_05SD':
    mua_threshold = (MUA & {'nwb_file_name': nwb_copy_file_name,
                 'interval_list_name':session_name}).fetch1('mean') + 0.5 * (MUA & {'nwb_file_name': nwb_copy_file_name,
                 'interval_list_name':session_name}).fetch1('sd')
elif decode_threshold_method == 'MUA_M05SD':
    mua_threshold = (MUA & {'nwb_file_name': nwb_copy_file_name,
                 'interval_list_name':session_name}).fetch1('mean') - 0.5 * (MUA & {'nwb_file_name': nwb_copy_file_name,
                 'interval_list_name':session_name}).fetch1('sd')
else:
    mua_threshold = 0

## find tetrodes with signal
groups_with_cell=(SpikeSortingRecordingSelection & {
        'nwb_file_name' : nwb_copy_file_name}).fetch('sort_group_id')
groups_with_cell=np.setdiff1d(groups_with_cell,[100,101])
channel_IDs = list(neural_df.keys())
    
spikeColInd = {}
for g in groups_with_cell:
    spikeColInd_ = np.argwhere(np.isin(channel_IDs,(Electrode() &  {'nwb_file_name' : nwb_copy_file_name,
                                                     'electrode_group_name':str(g)}).fetch('electrode_id'))).ravel()
    spikeColInd[g] = spikeColInd_

In [111]:
# Pre-make result directory:
decoding_path = (DecodeResultsLinear() & key_lk).fetch('posterior')[0]
savefolder_ss = os.path.join(os.path.split(decoding_path)[0],nwb_file_name,'SWR',epoch_name,'statespace')
savefolder_lk = os.path.join(os.path.split(decoding_path)[0],nwb_file_name,'SWR',epoch_name,'likelihood')
isExist = os.path.exists(savefolder_lk)
if not isExist:
    os.makedirs(savefolder_lk)

isExist = os.path.exists(savefolder_ss)
if not isExist:
    os.makedirs(savefolder_ss)

In [ ]:
for t in log_df.index[4:]:
    # Home
    ripples = log_df.loc[t].ripple_H
    ripple_table_inds = log_df.loc[t].ripple_ID_H

    if not np.isnan(log_df.loc[t].timestamp_H):
        # run time
        linear_position_subset = select_subset_helper(linear_position_df,[log_df.loc[t].timestamp_H, log_df.loc[t].timestamp_O])
        t0 = int(linear_position_subset.time[linear_position_subset.linear_position >= 20][0])
        t1 = log_df.loc[t].timestamp_O
        
        plottimes = [t0, t1]
        savename = 'trial' + str(t) + '_whole_trial_running' + nwb_copy_file_name[:-5] + '_' + session_name + '_likelihood'
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                             linear_position_df,
                             decode_lk,
                             spikeColInd,lfp_df,theta_df,
                             neural_df,mua_xr,head_speed,head_orientation,
                             savename,savefolder_lk,likelihood=True,wholeTrial = 1,plottimes=plottimes,mua_thresh = mua_thresh)

        savename = 'trial' + str(t) + '_whole_trial_running' + nwb_copy_file_name[:-5] + '_' + session_name + '_statespace'
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                             linear_position_df,
                             decode_ss,
                             spikeColInd,lfp_df,theta_df,
                             neural_df,mua_xr,head_speed,head_orientation,
                             savename,savefolder_lk,likelihood=False,wholeTrial = 1,plottimes=plottimes,mua_thresh = mua_thresh)

        savename = 'trial' + str(t) + '_whole_trial_home' + nwb_copy_file_name[:-5] + '_' + session_name + '_likelihood'
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                             linear_position_df,
                             decode_lk,
                             spikeColInd,lfp_df,theta_df,
                             neural_df,mua_xr,head_speed,head_orientation,
                             savename,savefolder_lk,likelihood=True,wholeTrial = 1,plottimes = [],mua_thresh = mua_thresh)

        savename = 'trial' + str(t) + '_whole_trial_home' + nwb_copy_file_name[:-5] + '_' + session_name + '_statespace'
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                             linear_position_df,
                             decode_ss,
                             spikeColInd,lfp_df,theta_df,
                             neural_df,mua_xr,head_speed,head_orientation,
                             savename,savefolder_lk,likelihood=False,wholeTrial = 1,plottimes = [],mua_thresh = mua_thresh)
    
            
    savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name + '_likelihood'
    plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                         linear_position_df,
                         decode_lk,
                         spikeColInd,lfp_df,theta_df,
                         neural_df,mua_xr,head_speed,head_orientation,
                         savename,savefolder_lk,likelihood=True,wholeTrial = 0, mua_thresh = mua_thresh)


    savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name + '_statespace_' + key_lk["classifier_param_name"]
    plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                         linear_position_df,
                         decode_lk,
                         spikeColInd,lfp_df,theta_df,
                         neural_df,mua_xr,head_speed,head_orientation,
                         savename,savefolder_lk,likelihood=False,wholeTrial = 0, mua_thresh = mua_thresh)


    savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name + '_statespace'
    plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                         linear_position_df,
                         decode_ss,
                         spikeColInd,lfp_df,theta_df,
                         neural_df,mua_xr,head_speed,head_orientation,
                         savename,savefolder_lk,likelihood=False,wholeTrial = 0, mua_thresh = mua_thresh)


/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_start_time = (~series.shift(1).fillna(False)) & series
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_end_time = series & (~series.shift(-1).fillna(False))
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ff

no ripples during this time. trial6_whole_trial_runninglewis20240109_02_Rev2Session1_likelihood


/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_start_time = (~series.shift(1).fillna(False)) & series
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_end_time = series & (~series.shift(-1).fillna(False))


no ripples during this time. trial6_whole_trial_runninglewis20240109_02_Rev2Session1_statespace


/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_start_time = (~series.shift(1).fillna(False)) & series
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_end_time = series & (~series.shift(-1).fillna(False))
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ff

...

/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_start_time = (~series.shift(1).fillna(False)) & series
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_end_time = series & (~series.shift(-1).fillna(False))
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ff

no ripples during this time. trial18_whole_trial_runninglewis20240109_02_Rev2Session1_likelihood


/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_start_time = (~series.shift(1).fillna(False)) & series
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_end_time = series & (~series.shift(-1).fillna(False))


no ripples during this time. trial18_whole_trial_runninglewis20240109_02_Rev2Session1_statespace


/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_start_time = (~series.shift(1).fillna(False)) & series
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_end_time = series & (~series.shift(-1).fillna(False))
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/ripple_detection/core.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ff

In [ ]:

# trial 8 20221021
# trial 42 20221021
# trial 51 20221021
# trial 57 20221021, in this example, you can also see the previous replay has multiple arms.
# trial 59 20221021
# trial 62 20221021
# trial 65 20221021

In [79]:
%debug

> /home/shijiegu/Documents/spyglass/src/spyglass/shijiegu/ripple_add_replay.py(552)plot_decode_spiking()
    550     if len(t0t1) > 0:
    551         t0t1_ = t0t1.ravel()
--> 552         first_shade_time = t0t1_[np.argwhere(t0t1_ >= plottimes[0]).ravel()[0]]
    553         second_shade_time = t0t1_[np.argwhere(t0t1_ <= plottimes[1]).ravel()[-1]]
    554         axes[0].set_title(title+'\n'+'first shade start time (s):'+str(first_shade_time)+'\n'+'last shade end time (s):'+str(second_shade_time),size=15)



ipdb>  t0t1_


array([1.66638300e+09, 1.66638300e+09, 1.66638301e+09, 1.66638301e+09])


ipdb>  exit


## End here

In [78]:
animal = 'Molly'
dates_to_plot = ['20220416','20220417','20220418','20220419','20220420']

encoding_set = '2Dheadspeed_above_4'
decode_threshold_method = 'MUA_05SD'

for d in dates_to_plot:
    nwb_file_name = animal.lower() + d + '.nwb'
    nwb_copy_file_name = animal.lower() + d + '_.nwb'
    #run_session_ids, run_session_names, pos_session_names = load_run_sessions(nwb_copy_file_name)

    all_epochs = list((TrialChoice() & {'nwb_file_name':nwb_copy_file_name}).fetch('epoch'))
    for epoch_num in all_epochs:
        (epoch_name,log_df,ripple_times,decode,
         head_speed,head_orientation,linear_position_df,
         lfp_df,theta_df,ripple_df,neural_df,mua_df) = replay_parser_master(nwb_file_name,epoch_num,
                                                                            encoding_set = encoding_set,
                                                                            decode_threshold_method = decode_threshold_method,
                                                                            extended = False)
    

[09:42:13][WARNING] Spyglass: Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.
03-May-25 09:42:13 Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.


epoch name 02_Seq2Session1
epoch name 02_Seq2Session1
epoch_pos_name pos 1 valid times


[2025-05-03 09:42:14,335][WARNING]: Skipped checksum for file with hash: c5f2b059-9845-dd98-1ffc-1428a999ef22, and path: /stelmo/nwb/analysis/molly20220416_XWNZWNKEUT.nwb
03-May-25 09:42:14 Skipped checksum for file with hash: c5f2b059-9845-dd98-1ffc-1428a999ef22, and path: /stelmo/nwb/analysis/molly20220416_XWNZWNKEUT.nwb
[2025-05-03 09:42:14,361][WARNING]: Skipped checksum for file with hash: c5f2b059-9845-dd98-1ffc-1428a999ef22, and path: /stelmo/nwb/analysis/molly20220416_XWNZWNKEUT.nwb
03-May-25 09:42:14 Skipped checksum for file with hash: c5f2b059-9845-dd98-1ffc-1428a999ef22, and path: /stelmo/nwb/analysis/molly20220416_XWNZWNKEUT.nwb
[2025-05-03 09:42:14,370][WARNING]: Skipped checksum for file with hash: c5f2b059-9845-dd98-1ffc-1428a999ef22, and path: /stelmo/nwb/analysis/molly20220416_XWNZWNKEUT.nwb
03-May-25 09:42:14 Skipped checksum for file with hash: c5f2b059-9845-dd98-1ffc-1428a999ef22, and path: /stelmo/nwb/analysis/molly20220416_XWNZWNKEUT.nwb
/home/shijiegu/anaconda3/

Done parsing. Insert into Spyglass
epoch name 04_Seq2Session2
epoch name 04_Seq2Session2
epoch_pos_name pos 3 valid times


[2025-05-03 09:48:53,974][WARNING]: Skipped checksum for file with hash: dca66c58-c248-239a-68d0-76fbb7aa2513, and path: /stelmo/nwb/analysis/molly20220416_0U86ASQYZZ.nwb
03-May-25 09:48:53 Skipped checksum for file with hash: dca66c58-c248-239a-68d0-76fbb7aa2513, and path: /stelmo/nwb/analysis/molly20220416_0U86ASQYZZ.nwb
[2025-05-03 09:48:53,998][WARNING]: Skipped checksum for file with hash: dca66c58-c248-239a-68d0-76fbb7aa2513, and path: /stelmo/nwb/analysis/molly20220416_0U86ASQYZZ.nwb
03-May-25 09:48:53 Skipped checksum for file with hash: dca66c58-c248-239a-68d0-76fbb7aa2513, and path: /stelmo/nwb/analysis/molly20220416_0U86ASQYZZ.nwb
[2025-05-03 09:48:54,006][WARNING]: Skipped checksum for file with hash: dca66c58-c248-239a-68d0-76fbb7aa2513, and path: /stelmo/nwb/analysis/molly20220416_0U86ASQYZZ.nwb
03-May-25 09:48:54 Skipped checksum for file with hash: dca66c58-c248-239a-68d0-76fbb7aa2513, and path: /stelmo/nwb/analysis/molly20220416_0U86ASQYZZ.nwb
/home/shijiegu/anaconda3/

...

Done parsing. Insert into Spyglass


[11:54:14][WARNING] Spyglass: Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.
03-May-25 11:54:14 Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.


epoch name 12_Seq2Session6
epoch name 12_Seq2Session6
epoch_pos_name pos 11 valid times


[2025-05-03 11:54:14,831][WARNING]: Skipped checksum for file with hash: 5483fbb3-1135-4c90-93da-5afa00407b02, and path: /stelmo/nwb/analysis/molly20220420_MALO38IAP2.nwb
03-May-25 11:54:14 Skipped checksum for file with hash: 5483fbb3-1135-4c90-93da-5afa00407b02, and path: /stelmo/nwb/analysis/molly20220420_MALO38IAP2.nwb
[2025-05-03 11:54:14,856][WARNING]: Skipped checksum for file with hash: 5483fbb3-1135-4c90-93da-5afa00407b02, and path: /stelmo/nwb/analysis/molly20220420_MALO38IAP2.nwb
03-May-25 11:54:14 Skipped checksum for file with hash: 5483fbb3-1135-4c90-93da-5afa00407b02, and path: /stelmo/nwb/analysis/molly20220420_MALO38IAP2.nwb
[2025-05-03 11:54:14,864][WARNING]: Skipped checksum for file with hash: 5483fbb3-1135-4c90-93da-5afa00407b02, and path: /stelmo/nwb/analysis/molly20220420_MALO38IAP2.nwb
03-May-25 11:54:14 Skipped checksum for file with hash: 5483fbb3-1135-4c90-93da-5afa00407b02, and path: /stelmo/nwb/analysis/molly20220420_MALO38IAP2.nwb
/home/shijiegu/anaconda3/

Done parsing. Insert into Spyglass


In [70]:
%debug

> /home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/datajoint/fetch.py(310)__call__()
    308             ret = cur.fetchone()
    309             if not ret or cur.fetchone():
--> 310                 raise DataJointError(
    311                     "fetch1 requires exactly one tuple in the input set."
    312                 )



ipdb>  u


> /home/shijiegu/Documents/spyglass/src/spyglass/shijiegu/ripple_add_replay.py(37)replay_parser_master()
     35     nwb_copy_file_name = get_nwb_copy_filename(nwb_file_name)
     36     # 1. Load state script
---> 37     key = (EpochPos & {'nwb_file_name':nwb_copy_file_name,'epoch':epoch_num}).fetch1()
     38     epoch_name = key['epoch_name']
     39     position_interval = key['position_interval']



ipdb>  key


*** NameError: name 'key' is not defined


...

ipdb>  EpochPos & {'nwb_file_name':nwb_copy_file_name,'epoch':epoch_num}


*nwb_file_name *epoch    epoch_name     position_inter
+------------+ +-------+ +------------+ +------------+

 (Total: 0)



ipdb>  EpochPos & {'nwb_file_name':nwb_copy_file_name}


*nwb_file_name *epoch    epoch_name     position_inter
+------------+ +-------+ +------------+ +------------+

 (Total: 0)



ipdb>  exit


#### with plots

In [ ]:
%matplotlib agg
all_epochs=list((TrialChoice() & {'nwb_file_name':nwb_copy_file_name}).fetch('epoch'))
for epoch_num in all_epochs:
    # load data

    """
    REAL WORK ENDS HERE, THE REST IS JUST PLOTTING
    """

    ## find tetrodes with signal
    groups_with_cell=(SpikeSortingRecordingSelection & {
        'nwb_file_name' : nwb_copy_file_name}).fetch('sort_group_id')
    groups_with_cell=np.setdiff1d(groups_with_cell,[100,101])
    channel_IDs = list(neural_df.keys())
    
    spikeColInd = {}
    for g in groups_with_cell:
        spikeColInd_ = np.argwhere(np.isin(channel_IDs,(Electrode() &  {'nwb_file_name' : nwb_copy_file_name,
                                                     'electrode_group_name':str(g)}).fetch('electrode_id'))).ravel()
        spikeColInd[g] = spikeColInd_

    # Pre-make result directory:
    decoding_path=(DecodeResultsLinear() & {'nwb_file_name':nwb_copy_file_name}).fetch('posterior')[0]
    exampledir=os.path.join(os.path.split(decoding_path)[0],nwb_file_name,'SWR',epoch_name)
    isExist = os.path.exists(exampledir)
    if not isExist:
        os.makedirs(exampledir)
    savefolder = exampledir
    print(savefolder)

    mua_thresh = (MUA & {'nwb_file_name' : nwb_copy_file_name,'interval_list_name':epoch_name}).fetch1('mean')


    for t in log_df.index[:-2]:
        # Home
        ripples = log_df.loc[t].ripple_H
        ripple_table_inds = log_df.loc[t].ripple_ID_H

        if not np.isnan(log_df.loc[t].timestamp_H):
            plottimes = [log_df.loc[t].timestamp_H,log_df.loc[t].timestamp_H + 6]
            savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name + '_whole_trial'
            plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                         linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                         neural_df,mua_df,head_speed,head_orientation,
                         savename,savefolder,wholeTrial = 1,plottimes=plottimes,mua_thresh = mua_thresh)
            
        savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                     linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                     neural_df,mua_df,head_speed,head_orientation,
                     savename,savefolder,wholeTrial = 0,mua_thresh = mua_thresh)
    
        
            
        # Outer Well
            
        ripples = log_df.loc[t].ripple_O
        ripple_table_inds = log_df.loc[t].ripple_ID_O

        plottimes = [log_df.loc[t].timestamp_O,log_df.loc[t].timestamp_O + 6]
        savename = 'trial' + str(t) + '_well_' + nwb_copy_file_name[:-5] + '_' + epoch_name + '_whole_trial'
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                     linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                     neural_df,mua_df,head_speed,head_orientation,
                     savename,savefolder,wholeTrial = 1,plottimes=plottimes,mua_thresh = mua_thresh)
        
        savename = 'trial' + str(t) + '_well_' + nwb_copy_file_name[:-5] + '_' + epoch_name
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                     linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                     neural_df,mua_df,head_speed,head_orientation,
                     savename,savefolder,wholeTrial = 0,mua_thresh = mua_thresh)
    
        

[15:56:46][WARNING] Spyglass: Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.
28-Oct-24 15:56:46 Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.


epoch name 02_Rev1Session1
epoch name 02_Rev1Session1
epoch_pos_name pos 1 valid times


[2024-10-28 15:56:48,245][WARNING]: Skipped checksum for file with hash: 942f52b0-a55f-b4dd-81cd-a9d9e0712783, and path: /stelmo/nwb/analysis/lewis20240118/lewis20240118_AK43PFM89I.nwb
28-Oct-24 15:56:48 Skipped checksum for file with hash: 942f52b0-a55f-b4dd-81cd-a9d9e0712783, and path: /stelmo/nwb/analysis/lewis20240118/lewis20240118_AK43PFM89I.nwb


In [135]:
((TetrodeNumber() & {'nwb_file_name': nwb_copy_file_name,"epoch_name":"02_Rev2Session1"})).fetch1("ca1_tetrode_ind")

array([  0,  14,  26,  46,  54,  66,  70,  95, 103, 107, 119, 124])

## Ends here

In [ ]:
"""
IMPORTANT!!!! RUN THIS FUNCTION EVERY TIME AFTER A NEW SESSION
USE OF GLOBAL VAR
"""

## find tetrodes with signal
groups_with_cell=(SpikeSortingRecordingSelection & {
    'nwb_file_name' : nwb_copy_file_name}).fetch('sort_group_id')
groups_with_cell=np.setdiff1d(groups_with_cell,[100,101])
channel_IDs = list(neural_df.keys())

spikeColInd = {}
for g in groups_with_cell:
    spikeColInd_ = np.argwhere(np.isin(channel_IDs,(Electrode() &  {'nwb_file_name' : nwb_copy_file_name,
                                                 'electrode_group_name':str(g)}).fetch('electrode_id'))).ravel()
    spikeColInd[g] = spikeColInd_

In [ ]:
spikeColInd

In [ ]:
(epoch_name,log_df,decode,head_speed,head_orientation,linear_position_df,
        lfp_df,theta_df,ripple_df,neural_df,mua_df) = load_epoch_data(nwb_copy_file_name,epoch_num)
ripple_times = pd.DataFrame((RippleTimes() & {'nwb_file_name': nwb_copy_file_name,
                                              'interval_list_name': epoch_name}).fetch1('ripple_times'))

In [ ]:
"""
IMPORTANT!!!! RUN THIS FUNCTION EVERY TIME AFTER A NEW SESSION
USE OF GLOBAL VAR
"""

def plot_ripples(ripples,ripple_table_inds,save_name,wholeTrial = 1):
    
    ripples = [r for r in ripples if len(r)>0]
    for ripple_ind in range(len(ripples)):
        t0t1 = np.vstack(ripples) #all possible shadings, just np.vstack(ripples), remove empty segments
        ripple_table_ind = ripple_table_inds[ripple_ind]
        
        if wholeTrial:
            plottimes = [t0t1.ravel()[0]-1,t0t1.ravel()[-1]+1]
            title = f"{save_name} all ripples"
            savename = save_name
        else:
            savename = save_name + str(ripple_table_ind)
            current_t0t1 = ripples[ripple_ind]
            plottimes = [current_t0t1.ravel()[0]-1,current_t0t1.ravel()[-1]+1]
    
            duration = int(ripple_times.loc[ripple_table_ind].duration * 1000)
            max_zscore = round(ripple_times.loc[ripple_table_ind].max_zscore,2)
            title = f"{savename} \n duration: {duration} ms, max_zscore: {max_zscore}"
            title += '\n Decoded arms:' + str(ripple_times.loc[ripple_table_ind].cont_intvl_replay)
    
        plot_decode_spiking(plottimes,t0t1,linear_position_df,decode,lfp_df,theta_df,
              neural_df,mua_df,ripple_df,head_speed,head_orientation,
              ripple_consensus_trace=None,
              title=title,savefolder=savefolder,savename=savename,
              simple=True,tetrode2ind = spikeColInd)
        if wholeTrial:
            break
    

In [ ]:
# Pre-make result directory:
decoding_path=(Decode() & {'nwb_file_name':nwb_copy_file_name}).fetch('posterior')[0]
exampledir=os.path.join(os.path.split(decoding_path)[0],nwb_file_name,'SWR',epoch_name)
isExist = os.path.exists(exampledir)
if not isExist:
    os.makedirs(exampledir)
savefolder = exampledir
print(savefolder)

In [ ]:
for t in log_df.index[:-2]:

    # Home
    ripples = log_df.loc[t].ripple_H
    ripple_table_inds = log_df.loc[t].ripple_ID_H
    savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name
    plot_ripples(ripples,ripple_table_inds,savename,wholeTrial = 0)
    plot_ripples(ripples,ripple_table_inds,savename,wholeTrial = 1)
    
    # Outer Well
    
    ripples = log_df.loc[t].ripple_O
    ripple_table_inds = log_df.loc[t].ripple_ID_O
    savename = 'trial' + str(t) + '_well_' + nwb_copy_file_name[:-5] + '_' + epoch_name
    plot_ripples(ripples,ripple_table_inds,savename,wholeTrial = 0)
    plot_ripples(ripples,ripple_table_inds,savename,wholeTrial = 1)


In [ ]:
#plt.plot(theta_t,theta_d)

### Plotting

In [ ]:
%%capture stored_output
add_replay(2)
add_replay(4)
add_replay(6)
add_replay(8)

### for whole day

In [ ]:
all_epochs=list((TrialChoice() & {'nwb_file_name':nwb_copy_file_name}).fetch('epoch'))
all_epochs

In [ ]:
%%capture stored_output
with multiprocessing.Pool(3) as p:
    p.map(add_replay,all_epochs)

In [ ]:
stored_output.show()

In [ ]:
print('here')

In [ ]:
add_replay(8)

In [ ]:
%%capture stored_output
add_replay(2)
add_replay(4)
add_replay(6)
add_replay(8)
add_replay(10)

In [ ]:
add_replay(10)
add_replay(12)

In [ ]:
    epoch_num=8
    
    # 1. Load state script
    key={'nwb_file_name':nwb_copy_file_name,'epoch':epoch_num}
    log=(TrialChoice & key).fetch1('choice_reward')
    epoch_name=(TrialChoice & key).fetch1('epoch_name')
    epoch_pos_name='pos '+str(int(epoch_name[:2])-1)+' valid times'
    print('epoch name',epoch_name)
    log_df=pd.DataFrame(log)

In [ ]:
log_df.loc[66,:]

In [ ]:
log_df.loc[66,:].timestamp_H

In [ ]:
linear_position_df = (IntervalLinearizedPosition() &
                      {'nwb_file_name': nwb_copy_file_name,
                       'interval_list_name': epoch_pos_name,
                       'position_info_param_name': 'default_decoding'}
                         ).fetch1_dataframe()

In [ ]:
    t=68
    
    # for each trial, find important times
    t_home=log_df.loc[t,'timestamp_H']
    t_outer=log_df.loc[t,'timestamp_O']
    if t+1<len(log_df):
        t_end=log_df.loc[t+1,'timestamp_H']
    else:
        t_end=log_df.loc[t,'timestamp_H']+10
    
    ''' HOME '''
    # find times close to home well
    time_slice = slice(t_home,t_outer)
    current_location = node_location['home']
    position_subset=linear_position_df.loc[time_slice]
    close_ind=np.argwhere(np.abs(position_subset.linear_position.values-
                                 current_location)<5).ravel()

In [ ]:
position_subset.index[0]

In [ ]:
position_subset

In [ ]:
    epoch_num=6
    
    # 1. Load state script
    key={'nwb_file_name':nwb_copy_file_name,'epoch':epoch_num}
    log=(TrialChoice & key).fetch1('choice_reward')
    epoch_name=(TrialChoice & key).fetch1('epoch_name')
    epoch_pos_name='pos '+str(int(epoch_name[:2])-1)+' valid times'
    print('epoch name',epoch_name)
    log_df=pd.DataFrame(log)

    # 1.5 Pre-make result directory:
    decoding_path=(Decode() & {'nwb_file_name':nwb_copy_file_name}).fetch('posterior')[0]
    exampledir=os.path.join(os.path.split(decoding_path)[0],nwb_file_name,epoch_name)
    isExist = os.path.exists(exampledir)
    if not isExist:
        os.makedirs(exampledir)

    # 2. Pre-expand result: Augment choice_reward table to be choice_reward_replay
    log_df_replay=log_df.copy()
    log_df_replay.insert(5,'ripple_H',[[] for i in range(len(log_df))]) #hold ripple times
    log_df_replay.insert(6,'ripple_O',[[] for i in range(len(log_df))])
    log_df_replay.insert(7,'replay_H',[[] for i in range(len(log_df))]) #hold decoded replay
    log_df_replay.insert(8,'ripple_peak_H',[[] for i in range(len(log_df))]) #hold decoded replay
    log_df_replay.insert(9,'replay_O',[[] for i in range(len(log_df))])
    log_df_replay.insert(10,'ripple_peak_O',[[] for i in range(len(log_df))]) #hold decoded replay

    # 3. linear map
    linear_map,node_location=get_linearization_map()

    # 4. load position, decoding, ripple times, mua threshold
    (linear_position_df,decode,ripple_nwb,
     ripple_timestamps,ripple_times,
     recordings,neural_ts,
     mua,mua_time)=load_everything(nwb_copy_file_name,
                    epoch_name,
                    epoch_pos_name)
    


In [ ]:
(IntervalList & {'nwb_file_name': nwb_copy_file_name,'interval_list_name': epoch_pos_name}).fetch1('valid_times')[0]

In [ ]:
    position_info = (IntervalPositionInfo() & {'nwb_file_name': nwb_copy_file_name,
                     'interval_list_name': epoch_pos_name,
                     'position_info_param_name': 'default'}).fetch1_dataframe()

In [ ]:
log_df.loc[60].timestamp_H

In [ ]:
log_df.loc[61].timestamp_H

In [ ]:
log_df.loc[82].timestamp_H

In [ ]:
ripple_timestamps[0]

In [ ]:
position_info.index[0]

In [ ]:
plt.plot(linear_position_df.loc[linear_position_df.index>1650309121,'linear_position'])

In [ ]:
add_replay(10)

In [ ]:
TrialChoiceReplay() & {'nwb_file_name':nwb_copy_file_name}

In [ ]:
choice_reward_replay=(TrialChoiceReplay() & {'nwb_file_name':nwb_copy_file_name,
                                            'epoch':2}).fetch1('choice_reward_replay')

In [ ]:
pd.DataFrame(choice_reward_replay)

### one trial

In [ ]:
mua_threshold=np.mean(mua)
mua_threshold

### all trials

In [ ]:
numWorker=30
print('num of worker:',numWorker)
with multiprocessing.Pool(numWorker) as p:
    results=p.map(find_ripple_classify_decode,legal_trials)

In [ ]:
len(results)

In [ ]:
for i in range(len(legal_trials)):
    t=legal_trials[i]
    # add to the choice_reward_replay
    log_df_replay.at[t, 'ripple_H']=results[i][0]
    log_df_replay.at[t, 'ripple_O']=results[i][3]
    
    log_df_replay.at[t, 'replay_H']= results[i][1]
    log_df_replay.at[t, 'replay_O']= results[i][4]
    
    log_df_replay.at[t, 'ripple_peak_H']= results[i][2]
    log_df_replay.at[t, 'ripple_peak_O']= results[i][5]
log_df_replay_subset=log_df_replay.loc[legal_trials,:].copy()

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
log_df_replay

In [ ]:
# insert into Spyglass
key={'nwb_file_name':nwb_copy_file_name,'epoch':epoch_num,'choice_reward_replay':log_df_replay_subset.to_dict()}
TrialChoiceReplay().make(key,replace=True)

In [ ]:
TrialChoiceReplay()

## investigate short ripples

In [ ]:
short_ripple_H=[]
for i in range(len(legal_trials)):
    t=trial_ind[i]
    ripple_H=log_df_replay.loc[t, 'ripple_H']
    for ri in range(len(ripple_H)):
        if len(ripple_H[ri])==0: #state time is short
            continue
        ripple_=np.reshape(ripple_H[ri],(1,-1))
        if (ripple_[0,-1]-ripple_[0,0])<=0.05:
            short_ripple_H.append((t,ri+1))

In [ ]:
replay_ripple_count=np.zeros((10,5))
replay_ripple=np.zeros((10,5),dtype=np.object)
for i in range(len(trial_ind)):
    t=trial_ind[i]
    replay_all=log_df_replay.loc[t,'replay_O']
    ripple_all=log_df_replay.loc[t,'ripple_peak_O']
    
    for ri in range(len(replay_all)):
        replay=replay_all[ri]
        unique_arms=np.unique(np.setdiff1d(replay,[5]))
        replay_arm_num=len(unique_arms)
    
        ripple_num=len(ripple_all[ri])
        if ripple_num<=10 and replay_arm_num<=5:
            replay_ripple_count[ripple_num-1,replay_arm_num-1]+=1
            if type(replay_ripple[ripple_num-1,replay_arm_num-1])=='list':
                replay_ripple[ripple_num-1,replay_arm_num-1].append(replay)
            else:
                replay_ripple[ripple_num-1,replay_arm_num-1]=replay

In [ ]:
replay_ripple[:,2]

In [ ]:
fig, axes = plt.subplots(nrows=1)
plt.imshow(np.flipud(replay_ripple_count.T))
axes.set_xlabel('ripple peak number')
axes.set_ylabel('distinct arms in replay')
axes.set_yticks([2,3,4])
axes.set_yticklabels(['3','2','1'])